<a href="https://colab.research.google.com/github/pauarmentaa/Regresion-No-Lineal-y-Regresion-Logistica/blob/main/a%C3%B1o2023_act7_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df= pd.read_csv("Gastos_costos_2023.csv")
df.head(2)

,Unnamed: 0,Fecha,Folio,Proveedor,Descripción,TIPO GASTO,Importe,IVA,TOTAL MX
0,0,2023-01-01,31,AESA ARTICULOS ELECTRICOS,Pago,COMPRAS,0.0,0.0,0.0
1,1,2023-01-01,38,AESA ARTICULOS ELECTRICOS,Pago,COMPRAS,0.0,0.0,0.0


In [ ]:
df["TIPO GASTO"].unique()

array(['COMPRAS', 'ARRENDAMIENTO FINANCIERO', 'COMISION BANCARIA',
       'CAPACITACION', 'SEGUROS', 'NO DEDUCIBLE', 'PUBLICIDAD',
       'TELEFONO', 'MTTO COMPUTO', 'COMISION MIXTA', 'HONORARIOS PM',
       'UNIFORMES', 'MONITOREO CAMARAS ADT', 'COMPRAS ', 'MENSAJERIA',
       'HONORARIOS CONTABLES', 'PAQUETERIA', 'ALIMENTOS',
       'MTTO TRANSPORTE', 'VARIOS', 'GERENCIA ADMON', 'DERECHOS',
       'RENTA COPIADORA', 'SUSCRIPCIONES', 'GASOLINA', 'MTTO LOCAL',
       'CREDITO ', 'MAQUINARIA', 'PAPELERIA', 'IMSS/INFONAVIT',
       'SUELDOS DIRECCION', 'MTTO MAQUINARIA', 'VALES DESPENSA',
       'MTTO OFICINA ', 'MTT OFICINA', 'Desconocido', 'RENTA AUTO',
       'FLETES'], dtype=object)

In [ ]:
#Convierto una variable a dicotómica 
df['TIPO GASTO']= df['TIPO GASTO'].replace([ 'ARRENDAMIENTO FINANCIERO', 'COMISION BANCARIA',
       'CAPACITACION', 'SEGUROS', 'NO DEDUCIBLE', 'PUBLICIDAD',
       'TELEFONO', 'MTTO COMPUTO', 'COMISION MIXTA', 'HONORARIOS PM',
       'UNIFORMES', 'MONITOREO CAMARAS ADT', 'COMPRAS ', 'MENSAJERIA',
       'HONORARIOS CONTABLES', 'PAQUETERIA', 'ALIMENTOS',
       'MTTO TRANSPORTE', 'VARIOS', 'GERENCIA ADMON', 'DERECHOS',
       'RENTA COPIADORA', 'SUSCRIPCIONES', 'GASOLINA', 'MTTO LOCAL',
       'CREDITO ', 'MAQUINARIA', 'PAPELERIA', 'IMSS/INFONAVIT',
       'SUELDOS DIRECCION', 'MTTO MAQUINARIA', 'VALES DESPENSA',
       'MTTO OFICINA ', 'MTT OFICINA', 'Desconocido', 'RENTA AUTO',"FLETES"], "OTRAS COMPRAS")
df

,Unnamed: 0,Fecha,Folio,Proveedor,Descripción,TIPO GASTO,Importe,IVA,TOTAL MX
0,0,2023-01-01,31,AESA ARTICULOS ELECTRICOS,Pago,COMPRAS,0.00,0.00,0.00
1,1,2023-01-01,38,AESA ARTICULOS ELECTRICOS,Pago,COMPRAS,0.00,0.00,0.00
2,2,2023-01-01,341932,BEPENSA LEASING S.A. DE C.V.,AA RENTA EXIGIBLE CL0000006876 30/36 de 03/12/...,OTRAS COMPRAS,5551.38,888.22,6439.60
3,3,2023-01-01,341933,BEPENSA LEASING S.A. DE C.V.,AA RENTA EXIGIBLE CL0000006877 30/36 de 03/12/...,OTRAS COMPRAS,5551.98,888.32,6440.30
4,4,2023-01-01,342205,BEPENSA LEASING S.A. DE C.V.,AA RENTA EXIGIBLE CL0000009486 5/48 de 03/12/2...,OTRAS COMPRAS,8454.37,1352.70,9807.07
...,...,...,...,...,...,...,...,...,...
392,392,2023-02-01,882974,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP...",COMISIONES E INTERESES,OTRAS COMPRAS,0.00,0.00,0.00
393,393,2023-02-01,053560,BEBIDAS PURIFICADAS,Epura 20 Lts Policarbonato,OTRAS COMPRAS,220.00,0.00,220.00
394,394,2023-02-01,1092957089,BANCO MERCANTIL DEL NORTE SA INSTITUCION DE BA...,SERVICIOS BANCARIOS,OTRAS COMPRAS,18071.52,682.40,18753.92
395,395,2023-02-01,1453820,EFECTIVALE S. de R.L. de C.V.,CONSUMOS DE COMBUSTIBLE DE MONEDEROS ELECTRONICOS,OTRAS COMPRAS,14472.32,0.00,14472.32


In [ ]:
df["TIPO GASTO"].unique()

array(['COMPRAS', 'OTRAS COMPRAS'], dtype=object)

In [ ]:
#Cargar archivo csv desde seaborn
#df=sns.load_dataset(name='titanic')
#df.head(20)

In [ ]:
#Rellenamos nulos
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")


In [ ]:
#Declaramos las variables dependientes e independientes para la regresión Logística
Vars_Indep= df[['TOTAL MX']]
Var_Dep= df['TIPO GASTO']

In [ ]:
#Redefinimos las variables 
X= Vars_Indep
y= Var_Dep

In [ ]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state =None)

In [ ]:
#Se escalan todos los datos
escalar = StandardScaler()

In [ ]:
#Para realizar el escalamiento de las variables “X” tanto de entrenamiento como de prueba, utilizaremos fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [ ]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [ ]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [ ]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test) 
y_pred

array(['OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'COMPRAS',
       'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS', 'OTRAS COMPRAS',
       'OTRAS COMPRAS', 'OTRAS 

In [ ]:
#Verifico la matriz de Confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 1 57]
 [ 2 60]]


In [ ]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Precisión del modelo:')
print(precision)

Precisión del modelo:
0.3333333333333333


In [ ]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.5083333333333333


In [ ]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.017241379310344827


In [ ]:
# el puntaje F1 que es una combinación entre la precisión y la sensibilidad, para esto importamos f1_score.
#Calculo el Puntaje F1 del modelo
from sklearn.metrics import f1_score

puntajef1 = f1_score(y_test, y_pred, average="binary", pos_label="COMPRAS")
print('Puntaje F1 del modelo:')
print(puntajef1)

Puntaje F1 del modelo:
0.03278688524590164


**Reemplazar valores**

In [ ]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['class'])
unico

array(['First', 'Second', 'Third'], dtype=object)